In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.getcwd() + '/core')
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import core.config as config
from core.model_loader import load_pretrained_VAE
from core.data_loader import TRAIN_loader, TEST_loader
from core.custom_loss import KL_div, VAE_loss
from core.fisher_utils import Calculate_Vanilafisher, Calculate_Vanilascore
from core.visualize import plot_hist, AUROC

In [2]:
train_dist = 'cifar10'
opt = config.VAE_cifar10
netE, netG = load_pretrained_VAE(option=train_dist)

Encoder(
  (conv1): Conv2d(512, 100, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(512, 100, kernel_size=(4, 4), stride=(1, 1))
  (main): Sequential(
    (input-conv): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (input-BN): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (input-relu): ReLU(inplace=True)
    (pyramid:128-256:conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid:256:batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid:256:relu): ReLU(inplace=True)
    (pyramid:256-512:conv): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid:512:batchnorm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pyramid:512:relu): ReLU(inplace=True)
  )
)

In [45]:
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms

netE.eval()
netG.eval()

for p in netE.parameters():
    p.requires_grad = False
for p in netG.parameters():
    p.requires_grad = False
for p in netE.conv1.parameters():
    p.requires_grad = True
for p in netE.conv2.parameters():
    p.requires_grad = True

optimizer1 = optim.Adam(netE.parameters(), lr=1e-4)
optimizer2 = optim.Adam(netG.parameters(), lr=1e-4)

device = 'cuda:0'
# for i, p in enumerate(netE.parameters()):
#     if i==0:
#         parameter_i = p.cpu().detach()
#         break
#parameter_i = netE.conv1.bias.cpu().detach()
#parameter_i = netE.conv1.weights.cpu().detach()

dataset = dset.CIFAR10( root=opt.dataroot, download=True,train = True,
                                transform=transforms.Compose([
                                   transforms.Resize((opt.imageSize)),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                               ]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                        shuffle=True, num_workers=int(opt.workers))

for epoch in range(1):
    for i, (x, _) in enumerate(dataloader):
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        x = x.to(device)
        [z,mu,logvar] = netE(x)
        recon = netG(z)
        loss = VAE_loss(x, [recon, mu, logvar], beta=opt.beta)
        loss.backward(retain_graph=True)
        optimizer1.step()
        optimizer2.step()

# for i, p in enumerate(netE.parameters()):
#     if i==0:
#         parameter_f = p.cpu().detach()
#         break
#parameter_f = netE.conv1.bias.cpu().detach()
#parameter_f = netE.conv1.weights.cpu().detach()

Files already downloaded and verified


In [50]:
print(torch.norm(parameter_f-parameter_i))

tensor(1.6996)
